In [ ]:
import pickle5 as pickle
with open('../../data/COVID-QA/top_N_ents_spacy-COVID_QA.pkl', 'rb') as f:
    top_N_ents = pickle.load(f)

In [ ]:
import wikipedia
from tqdm.auto import tqdm

search_res = {}

for i, ent in tqdm(enumerate(top_N_ents)):
    #Skipping those entities which don't return anything
    if wikipedia.search(ent) != []:
        search_res[ent] = wikipedia.search(str(ent), results=1)[0]

In [ ]:
import wikipedia
from transformers import AutoTokenizer, AutoModel
import torch

context_dict = {}

filtering = False
if filtering == True:
    filtering_threshold = 0.5
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    #We're consider scibert because pubmedbert assigns very high similarity for both related/unrelated terms.
    checkpoint = 'allenai/scibert_scivocab_uncased'
    
    model = AutoModel.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    cos = torch.nn.CosineSimilarity(dim=0)
    model.to(device)

for ent,res in tqdm(search_res.items()):
    if filtering == False:
        try:
            context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
        except:
            continue
    else:
        encoded_input = tokenizer([ent, res], return_tensors='pt', padding=True)
        with torch.no_grad():
            output = model(**encoded_input)
        
        similarity = cos(output.pooler_output[0], output.pooler_output[1])
        '''
        we're taking less than here since the similarity scores for related terms seem to be lower than 
        unrelated ones.
        '''
        if similarity.item() < filtering_thresold:
            try:
                context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
            except:
                continue

In [ ]:
import pandas as pd
pd.DataFrame(context_dict.items(), columns = ['ent', 'text']).to_parquet('wiki_corpus.parquet', index=False)

In [ ]:
from pymed import PubMed

# Create a PubMed object that GraphQL can use to query
# Note that the parameters are not required but kindly requested by PubMed Central
# https://www.ncbi.nlm.nih.gov/pmc/tools/developers/
pubmed = PubMed(tool="MyTool", email="my@email.address")

# Create a GraphQL query in plain text
query = ['covid-19', 'hiv-1']

# Execute the query against the API
results = pubmed.query(query, max_results=500)

# Loop over the retrieved articles
for article in results:

    # Extract and format information from the article
    article_id = article.pubmed_id
    title = article.title
    abstract = article.abstract

    # Show information about the article
    print(
        f'{abstract}\n'
    )

In [95]:
#Squadify!

import json
from datasets import load_dataset

input_filename = '../../data/RadQA'\
                 '/radqa-a-question-answering-dataset-to-improve-comprehension-of-radiology-reports-1.0.0'\
                 '/dev.json'
output_filename = '../../data/RadQA'\
                 '/radqa-a-question-answering-dataset-to-improve-comprehension-of-radiology-reports-1.0.0'\
                 '/dev.jsonl'

with open(input_filename, encoding="utf-8") as f:
    radqa = json.load(f)

with open(output_filename, "w", encoding="utf-8") as f:
    for example in radqa["data"]:
        title = example.get("title", "")
        for paragraph in example["paragraphs"]:
            context = paragraph["context"]  # do not strip leading blank spaces GH-2585
            for qa in paragraph["qas"]:
                question = qa["question"]
                id_ = qa["id"]

                answer_starts = [answer["answer_start"] for answer in qa["answers"]]
                answers = [answer["text"] for answer in qa["answers"]]              
                f.write(
                    json.dumps(
                        {
                    "title": title,
                    "context": context,
                    "question": question,
                    "id": id_,
                    "answers": {
                        "answer_start": answer_starts,
                        "text": answers,
                               },
                        }
                               )
                       )
                f.write("\n")

#ds = load_dataset("json", data_files=output_filename)

In [ ]:
all_contexts = []
all_q = []
for row in s['train']['paragraphs']:
    for d in row:
        all_contexts.append(d["context"])
        for q in d['qas']:
            all_q.append(q["question"])

In [ ]:
import re
c=0
for r in all_contexts:
    #if re.search('findings',r, re.IGNORECASE) and re.search('impression:', r, re.IGNORECASE):
    #if re.search('final report', r, re.IGNORECASE) and re.search('impression:', r, re.IGNORECASE):
    if re.search('findings and impression', r, re.IGNORECASE):
        print(r)
        print('-'*100)
        

In [1]:
import pickle5 as pickle
with open('../../data/RadQA/ents_spacy_radqa.pkl', 'rb') as f:
    s = pickle.load(f)

In [188]:
import re
c=0
filtered=[]
for ent in s:
    if not re.search(r"\[|\]|\.|(patient['s]*)|baby|report|conclusion|finding*|impression*|procedure|\btime\b", ent, re.IGNORECASE) \
    and len(ent.strip()) >= 3:
        filtered.append(ent.replace('\n',''))
with open('ents_spacy_radqa_filtered.pkl', 'wb') as f:
    pickle.dump(filtered, f)

In [189]:
len(filtered)

10334

In [66]:
class QADataset(Dataset):
    def __init__(self, dataset_location):
        if dataset_location == 'remote':
            self.dataset = load_dataset('squad_v2')
        else:
            self.dataset = DatasetDict({"validation": load_dataset('json', data_files=os.path.abspath(
                '../../data/RadQA/radqa-a-question-answering'
                '-dataset-to-improve-comprehension-of'
                '-radiology-reports-1.0.0/dev.jsonl'))['train']})

        self.samples = self.dataset['validation']
        self.questions = self.samples['question']
        self.contexts = self.samples['context']

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return {'question': self.questions[idx], 'context': self.contexts[idx]}

In [71]:
import os
ds = QADataset('local')

Found cached dataset json (/home/saptarshi/.cache/huggingface/datasets/json/default-a680d41adda12e33/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

In [87]:
from transformers import default_data_collator
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

import torch, random
import numpy as np
from transformers import set_seed
g = torch.Generator()
g.manual_seed(42)
torch.manual_seed(42)
random.seed(42)
set_seed(42)


data_loader = DataLoader(ds, batch_size=16, shuffle=False,
                             worker_init_fn=seed_worker, generator=g)

In [88]:
len(data_loader)

41

In [90]:
from tqdm import tqdm
for batch in tqdm(data_loader):
    print(batch)
    break

  0%|                                                    | 0/41 [00:00<?, ?it/s]

{'question': ["Can the patient's shortness of breath be explained by any infiltrations in the lungs?", 'Is infil present?', "Can the patient's shortness of breath be explained by any infiltrations in the lungs?", 'Is infil present?', 'Was the staging complete?', 'Did the gastric cancer metastasize to chest?', 'Does the pelvis show any findings of gastric cancer metastasis?', 'Are there any abnormal lymph nodes suggesting spread of gastric cancer?', 'Is there evidence of mets?', 'Is there any abnormal tissue suggesting of gastric cancer in the adbomen?', 'Is there any abnormal tissue suggesting of gastric cancer in the adbomen?', 'Did the gastric cancer metastasize to chest?', 'Does the pelvis show any findings of gastric cancer metastasis?', 'Was the staging complete?', 'Is there evidence of mets?', 'Are there any abnormal lymph nodes suggesting spread of gastric cancer?'], 'context': ['IMPRESSION:  Right mid lung opacity is concerning for early pneumonia.', 'IMPRESSION:  Right mid lun